In [74]:
import numpy as np
import os
import random
import cv2
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense

In [75]:
batch_size = 2000
gt_all = os.listdir('gt')

X = np.zeros((batch_size,64,64,3))
y = np.zeros((batch_size))
for a in range(batch_size):
    i = random.randint(0,len(gt_all)-1)

    name = gt_all[i]
    name = name.replace('_segmentation','')
    name = name.replace('png','jpg')
    img = cv2.imread(os.path.join('gt',gt_all[i]))
    thresh = 127
    img = cv2.resize(img,(64,64))
    img = cv2.threshold(img, thresh, 1, cv2.THRESH_BINARY)[1]
    X[a] = img
    if name in os.listdir('melanoma'):
        img = cv2.imread(os.path.join('melanoma',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            X[a] = np.multiply(X[a],img)
            y[a] = 0
    else:
        img = cv2.imread(os.path.join('others',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            X[a] = np.multiply(X[a],img)
            y[a] = 1

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [80]:
num_classes = 1
batch_size = 50
epochs = 10

In [93]:
class_weights = [0.5, 2.2]

In [89]:
num_classes = 2
model = Sequential()

model.add(Convolution2D(20, 5, 5, border_mode="same", input_shape=(64, 64, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# second set of CONV => RELU => POOL
model.add(Convolution2D(50, 5, 5, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# set of FC => RELU layers
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

# softmax classifier
model.add(Dense(num_classes))
model.add(Activation("sigmoid"))



/anaconda3/envs/cv2/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (5, 5), input_shape=(64, 64, 3..., padding="same")`
/anaconda3/envs/cv2/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (5, 5), padding="same")`


In [82]:
yLabels = np_utils.to_categorical(y, 2)

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, yLabels, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [55]:
opt = SGD(lr=0.01)

In [90]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [58]:
print("[INFO] training...")
model.fit(X_train, y_train, batch_size=128, nb_epoch=20, verbose=1)

# show the accuracy on the testing set
print("[INFO] evaluating...")
(loss, accuracy) = model.evaluate(X_test, y_test, batch_size=128, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(accuracy * 100))


[INFO] training...
Epoch 1/20


/anaconda3/envs/cv2/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


1280/1280 [==============================] - 266s 208ms/step - loss: 3.2435 - acc: 0.7742
Epoch 2/20
1280/1280 [==============================] - 254s 199ms/step - loss: 2.9970 - acc: 0.8141
Epoch 3/20
1280/1280 [==============================] - 255s 199ms/step - loss: 2.9970 - acc: 0.8141
Epoch 4/20
1280/1280 [==============================] - 255s 200ms/step - loss: 2.9970 - acc: 0.8141
Epoch 5/20
1280/1280 [==============================] - 255s 199ms/step - loss: 2.9970 - acc: 0.8141
Epoch 6/20
1280/1280 [==============================] - 249s 195ms/step - loss: 2.9970 - acc: 0.8141
Epoch 7/20
1280/1280 [==============================] - 249s 195ms/step - loss: 2.9970 - acc: 0.8141
Epoch 8/20
1280/1280 [==============================] - 264s 206ms/step - loss: 2.9970 - acc: 0.8141
Epoch 9/20
1280/1280 [==============================] - 340s 265ms/step - loss: 2.9970 - acc: 0.8141
Epoch 10/20
1280/1280 [==============================] - 251s 196ms/step - loss: 2.9970 - acc: 0.8141


In [62]:
predicted = model.predict(X_test)

In [67]:
cm=confusion_matrix(y_test.argmax(axis=1), predicted.argmax(axis=1))

In [71]:
def get_fscore(cm):
    precision = cm[0][1]/(cm[1][1]+cm[0][1])
    recall =  cm[0][1]/(cm[0][1]+cm[0][0])

    f1 = 2*precision*recall/(precision+recall)
    print("Melanoma F1 score:",f1)
   
    precision = cm[1][1]/(cm[1][1]+cm[1][0])
    recall =  cm[1][1]/(cm[1][1]+cm[0][1])

    f1 = 2*precision*recall/(precision+recall)
    print("Non-melanoma F1 score:",f1)

In [72]:
get_fscore(cm)

Melanoma F1 score: 0.319327731092
Non-melanoma F1 score: 0.895027624309
